# Periodicity Study (Colab)

This notebook clones the repo, installs dependencies, and runs the full study on GPU:
- Offline CRTR + PPO with metrics (1/2/3) logged over time
- Online joint CRTR + bonus + PPO with metrics (1/2/3) logged over time

In [ ]:
import os
from pathlib import Path

REPO_URL = "https://github.com/jinschofield/invariance_aware.git"
REPO_DIR = Path("/content/invariance_aware")

if not REPO_DIR.exists():
    !git clone {REPO_URL} {REPO_DIR}
else:
    !git -C {REPO_DIR} pull

%cd /content/invariance_aware

!pip install -r requirements.txt
!pip install -e .

import torch
print("CUDA available:", torch.cuda.is_available())

FAST = False  # set True for a quick run

cmd = "python -m periodicity_study.run_study --device cuda:0"
if FAST:
    cmd += " --fast"

!{cmd}

from IPython.display import Image, display

fig_root = REPO_DIR / "periodicity_study" / "outputs" / "figures"
metric_keys = [
    "rep_invariance_mean",
    "bonus_within_std_mean",
    "bonus_between_std",
    "bonus_within_over_between",
    "action_kl_mean",
]

env_dirs = sorted([p for p in fig_root.iterdir() if p.is_dir()])
if not env_dirs:
    print(f"No per-env figure folders found in {fig_root}")
else:
    for env_dir in env_dirs:
        print(f"\n=== {env_dir.name} ===")
        for key in metric_keys:
            for suffix in ("", "_goal"):
                path = env_dir / f"timeseries_compare_{key}{suffix}.png"
                if path.exists():
                    print(path.name)
                    display(Image(filename=str(path)))
                else:
                    print(f"Missing: {path}")

In [ ]:
# Optional: run ONLY goal-based PPO for learned reps (offline + online)
# This skips coord-only reps and only runs the 4 learned rep PPOs with extrinsic reward.
cmd = (
    "python -m periodicity_study.run_study --device cuda:0 "
    "--goal-only --reps crtr_learned,idm_learned,crtr_online_joint,idm_online_joint"
)
!{cmd}
